In [11]:
import sqlite3 as sqlite

def cleanDB():
    with sqlite.connect(r'telegram.db') as con: 
        cur = con.cursor()
        
        cur.execute("drop table if exists users;")
        cur.execute("create table users(id integer primary key, first_name text, last_name text, username text unique, phone text unique)")
    
        cur.execute("drop table if exists conversations;")
        cur.execute("create table conversations(id integer primary key, name text, user_id integer, FOREIGN KEY (user_id) REFERENCES users(id))")
        
        cur.execute("drop table if exists chats;")
        cur.execute("create table chats(id integer primary key, from_id integer, created_at text, msg text, sentiment_score float, con_id integer, \
                    FOREIGN KEY (con_id) REFERENCES conversations(id))")
        
        con.commit()

def isUserExists(user_id):
    with sqlite.connect(r'telegram.db') as con: 
        cur = con.cursor()
        cur.execute("select * from users where id = " + str(user_id))
        rows = cur.fetchall()
        if(len(rows) > 0):
            return True
        else:
            return False

def addUser(user):
    with sqlite.connect(r'telegram.db') as con: 
        cur = con.cursor()
        cur.execute("insert or ignore into users(id,first_name,last_name,username,phone) values(?,?,?,?,?)", user)
        con.commit()
        
def addConversations(convs):
    with sqlite.connect(r'telegram.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into conversations(id,name,user_id) values(?,?,?)", convs)
        con.commit()

def addChats(chats):
    with sqlite.connect(r'telegram.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into chats(id,from_id,created_at,msg,sentiment_score,con_id) values(?,?,?,?,?,?)", chats)
        con.commit()

In [12]:
import logging
logging.basicConfig(format='[%(levelname) 5s/%(asctime)s] %(name)s: %(message)s',
                    level=logging.WARNING)

#cleanDB()

In [13]:
import os
os.environ['KERAS_BACKEND']='theano'
from keras.models import load_model
from models import AttLayer
import numpy as np
from tensorflow.contrib import learn

data = "formspring"
model_type = "blstm_attention"
vector_type = "sswe"
embed_size = "50"
output_folder_name = "results/"

vocab_filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + embed_size + "_vbp.pkl"
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_filename)
    
model_filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + embed_size + ".model"
model = load_model(model_filename, custom_objects={'AttLayer': AttLayer})

def getSentimentValue(test_x):
    testX = np.array(list(vocab_processor.transform([test_x])))
    temp = model.predict(testX)
    return temp[0][0]
    

In [14]:
from telethon import TelegramClient, events

api_id = 636602
api_hash = '2460690ad3e6cbeb7f0797256a0c76a7'

async def signInTelegram(username, phone):
    client = TelegramClient('sessions/' + username, api_id, api_hash)
    
    await client.start(phone)
    async with client:
        # Getting information about yourself
        me = await client.get_me()

        if(isUserExists(me.id) == False):
            addUser((me.id,me.first_name,me.last_name,me.username,me.phone))

            allDialogs = []
            allMessages = []
            # You can print all the dialogs/conversations that you are part of:
            dialogs = await client.get_dialogs()
            allDialogs += [(x.id, x.name, me.id) for x in dialogs]
            for dialog in dialogs:
                # You can print the message history of any chat:
                messages = await client.get_messages(dialog.id, limit=100)
                allMessages += [(x.id, x.from_id, x.date, x.message, 0, dialog.id) for x in messages if (x.message != None and x.message != '')]

            addConversations(allDialogs)
            addChats(allMessages)
            
        @client.on(events.NewMessage)
        async def my_event_handler(event):
            if(event.raw_text and event.raw_text != ''):
                sentiment = getSentimentValue(event.raw_text)
                #addChats([(event.id, event.sender_id, event.date, event.raw_text, sentiment, event.chat_id)])
                sentimentReply = ''
                score = str("%.2f" % (sentiment * 100)) + '%';
                
                if(sentiment <= 0.25):
                    sentimentReply = "\U0001F621 Negative Score: " + score  
                elif(sentiment >= 0.75):
                    sentimentReply = "\U0001F600 Positive Score: " + score
                else:
                    sentimentReply = "\U0001F633 Neutral Score: " + score
                
                await event.reply(sentimentReply)
                
        await client.start()
        await client.run_until_disconnected()
        
await signInTelegram('ZohaibK', '+4917681067652')

CancelledError: 